In [22]:
!pip install oletools

In [23]:
import pandas as pd

# Define the path to the folder in your Google Drive
folder_path = '/content/drive/MyDrive/ScrapedLinks'

# Define the filename of the CSV file in the folder
filename = 'all_results_excel.csv'

# Define the path to the CSV file in your Google Drive
csv_path = folder_path + '/' + filename

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(csv_path)

# Convert the DataFrame to a list (and flatten it)

result_list_excel = [item for sublist in df.values.tolist() for item in sublist]

In [24]:
import pandas as pd

# Define the path to the folder in your Google Drive
folder_path = '/content/drive/MyDrive/ScrapedLinks'

# Define the filename of the CSV file in the folder
filename = 'all_results_word.csv'

# Define the path to the CSV file in your Google Drive
csv_path = folder_path + '/' + filename

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(csv_path)

# Convert the DataFrame to a list (and flatten it)

result_list_word = [item for sublist in df.values.tolist() for item in sublist]

In [25]:
all_results = result_list_excel + result_list_word

In [28]:
from requests.exceptions import RequestException
import requests
import tempfile
from io import BytesIO
import os
import oletools
from oletools import olevba
import subprocess

count = 0
show_results = 0
results_macro_sus = []

for url in all_results:
    try:
        response = requests.get(url, timeout=5)
    except RequestException:
        pass

    if show_results % 25 == 0:
        print("gabbogabri", url)

    with tempfile.NamedTemporaryFile(delete=False) as temp_file:
        temp_file.write(response.content)
        temp_file_path = temp_file.name

    # Check for macros using olevba
    vba = olevba.VBA_Parser(temp_file_path)
    if vba.detect_vba_macros():
        print(f"Document {url} has macros embedded in it.")
        # Scan the VBA code using mraptor
        mraptor_result = subprocess.run(["mraptor", temp_file_path], capture_output=True)

        if mraptor_result.returncode == 20:
            print(f"Document {url} contains potentially malicious VBA code:\n{mraptor_result.stdout.decode()}")
            results_macro_sus.append(url)
        else:
            print(f"Document {url} does not contain potentially malicious VBA code.")

    else:
        all_results.remove(url)
        count += 1
        print(f"{count} deleted urls.")

    show_results += 1
    

    # Clean up the temp file
    os.remove(temp_file_path)

gabbogabri https://www.iasplus.com/en/publications/global/models-checklists/2019/ifrs-checklist/at_download/file/IFRS%2520Checklist%25202019.xlsm
Document https://www.iasplus.com/en/publications/global/models-checklists/2019/ifrs-checklist/at_download/file/IFRS%2520Checklist%25202019.xlsm has macros embedded in it.
Document https://www.iasplus.com/en/publications/global/models-checklists/2019/ifrs-checklist/at_download/file/IFRS%2520Checklist%25202019.xlsm contains potentially malicious VBA code:
MacroRaptor 0.56.2 - http://decalage.info/python/oletools
This is work in progress, please report issues at https://github.com/decalage2/oletools/issues
----------+-----+----+--------------------------------------------------------
Result    |Flags|Type|File                                                    
----------+-----+----+--------------------------------------------------------
WARNING  For now, VBA stomping cannot be detected for files in memory
SUSPICIOUS|A-X  |OpX:|/tmp/tmpbbbof09h

Document https://www.ebola.sante.gouv.fr/IMG/xlsm/grille_de_lecture_rse__cnis_20221210.xlsm has macros embedded in it.
Document https://www.ebola.sante.gouv.fr/IMG/xlsm/grille_de_lecture_rse__cnis_20221210.xlsm contains potentially malicious VBA code:
MacroRaptor 0.56.2 - http://decalage.info/python/oletools
This is work in progress, please report issues at https://github.com/decalage2/oletools/issues
----------+-----+----+--------------------------------------------------------
Result    |Flags|Type|File                                                    
----------+-----+----+--------------------------------------------------------
WARNING  For now, VBA stomping cannot be detected for files in memory
SUSPICIOUS|A-X  |OpX:|/tmp/tmpa7laq6ex                                        

Flags: A=AutoExec, W=Write, X=Execute
Exit code: 20 - SUSPICIOUS

Document https://www.schaffnerhof.ch/download/ws-multi-sprachlernkarten_2008a.xlsm has macros embedded in it.
Document https://www.schaffnerho

Elimina vecchia folder

In [31]:
import os

# Set the path to the folder in your Google Drive
folder_path = '/content/drive/MyDrive/Scraped_Links'

# Loop through all files in the folder and delete them
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path):
        os.remove(file_path)

Carica risultati puliti da file senza macro

In [32]:
import json
import pandas as pd

# Write results to JSON file
with open("all_results.json", 'w') as f:
    # indent=2 is not needed but makes the file human-readable 
    # if the data is nested
    json.dump(all_results, f, indent=2) 

# Read results from JSON file and convert to DataFrame
with open("all_results.json", 'r') as f:
    all_results = json.load(f)
df = pd.DataFrame(all_results)

# Define the path to the folder in your Google Drive
folder_path = '/content/drive/MyDrive/Scraped_Links'

# Save the DataFrame as a CSV file in your Google Drive
csv_path = folder_path + '/all_results.csv'
df.to_csv(csv_path, index=False)

print('CSV file saved to:', csv_path)

CSV file saved to: /content/drive/MyDrive/Scraped_Links/all_results.csv


In [34]:
import json
import pandas as pd

# Write results to JSON file
with open("results_macro_sus.json", 'w') as f:
    # indent=2 is not needed but makes the file human-readable 
    # if the data is nested
    json.dump(results_macro_sus, f, indent=2) 

# Read results from JSON file and convert to DataFrame
with open("results_macro_sus.json", 'r') as f:
    results_macro_sus = json.load(f)
df = pd.DataFrame(results_macro_sus)

# Define the path to the folder in your Google Drive
folder_path = '/content/drive/MyDrive/Scraped_Links'

# Save the DataFrame as a CSV file in your Google Drive
csv_path = folder_path + '/results_macro_sus.csv'
df.to_csv(csv_path, index=False)

print('CSV file saved to:', csv_path)

CSV file saved to: /content/drive/MyDrive/Scraped_Links/results_macro_sus.csv
